## Setup

Run this cell to install required packages (only needed on Google Colab).

In [ ]:
# Install dependencies (for Google Colab)
import sys
if 'google.colab' in sys.modules:
    !pip install -q rustworkx matplotlib numpy scipy

In [ ]:
# Import libraries
import random
import math
import os
import numpy as np
from scipy.stats import binom, poisson
import matplotlib.pyplot as plt
from itertools import combinations

import rustworkx as rx
from rustworkx.visualization import mpl_draw

# Visualization settings
NS_PURPLE = '#8e44ad'
NS_GREEN = '#2ecc71'
NS_ORANGE = '#FF9800'

plt.rcParams['figure.figsize'] = (8, 5)
plt.rcParams['figure.dpi'] = 100
plt.rcParams.update({
    'axes.spines.top': False,
    'axes.spines.right': False,
})

In [ ]:
def set_seed(seed=None):
    """Set random seeds for reproducibility."""
    if seed is None:
        seed = int.from_bytes(os.urandom(4), 'big')
    random.seed(seed)
    np.random.seed(seed)
    return seed

---
## 3.2 The Random Network Model

A **random network** (Erdős–Rényi model) consists of $N$ nodes where each pair of nodes is connected with probability $p$.

### Key Definitions

| Term | Description |
|------|-------------|
| $G(N, p)$ | Each pair of $N$ labeled nodes is connected independently with probability $p$ |
| $G(N, L)$ | $N$ labeled nodes are connected by $L$ randomly placed links |
| $N$ | Number of nodes in the network |
| $p$ | Probability of connection between any two nodes |

### G(N, p) Model

Each pair of nodes is connected with probability $p$.

In [ ]:
set_seed(2)

N = 5
p = 0.3

nodes = list(range(N))
possible_edges = [(u, v) for u, v in combinations(nodes, 2)]
edges = [e for e in possible_edges if random.random() < p]

print(f"Number of nodes: {N}")
print(f"Possible edges: {len(possible_edges)}")
print(f"Generated edges: {len(edges)} → {edges}")

### G(N, L) Model

Exactly $L$ links are randomly placed among all possible pairs.

In [ ]:
set_seed(3)

N = 5
L = 3

nodes = list(range(N))
all_possible = list(combinations(nodes, 2))
n_possible = len(all_possible)

# Randomly select L edges
selected_indices = np.random.choice(n_possible, L, replace=False)
edges = [all_possible[i] for i in selected_indices]

print(f"Number of nodes: {N}")
print(f"Possible edges: {n_possible}")
print(f"Selected {L} edges: {edges}")

### Visualizing Random Networks

Different realizations of the same $G(N, p)$ model produce different networks.

In [ ]:
set_seed(4)

N, p = 10, 0.3

# Generate three different realizations
graphs = [rx.undirected_gnp_random_graph(N, p) for _ in range(3)]

fig, axes = plt.subplots(1, 3, figsize=(12, 4))

for i, (g, ax) in enumerate(zip(graphs, axes)):
    layout = rx.circular_layout(g)
    mpl_draw(g, pos=layout, ax=ax, node_color=NS_PURPLE, edge_color=NS_GREEN,
             with_labels=False, node_size=400)
    ax.set_title(f"Realization {i+1}: {g.num_edges()} edges")
    ax.set_axis_off()

plt.suptitle(f"Three G(N={N}, p={p}) Random Networks", fontsize=14)
plt.tight_layout()
plt.show()

---
## 3.3 Number of Links

The probability that a random network has exactly $L$ links follows a binomial distribution:

$$p_L = \binom{\frac{N(N-1)}{2}}{L} p^L (1 - p)^{\frac{N(N-1)}{2} - L}$$

This is the product of:
- $\binom{\frac{N(N-1)}{2}}{L}$: Number of ways to place $L$ links among all pairs
- $p^L$: Probability that $L$ attempts result in links
- $(1-p)^{\frac{N(N-1)}{2} - L}$: Probability remaining attempts don't create links

In [ ]:
def prob_L_links(N, p, L):
    """Probability that a random network has exactly L links."""
    n_possible = math.comb(N, 2)  # N(N-1)/2
    n_ways = math.comb(n_possible, L)
    return n_ways * (p ** L) * ((1 - p) ** (n_possible - L))

# Example
N, p, L = 10, 0.1, 5
print(f"P(L={L}) = {prob_L_links(N, p, L):.4f}")

### Distribution of Link Counts

In [ ]:
N, p = 10, 0.3

n_possible = math.comb(N, 2)
Ls = list(range(n_possible + 1))
pmf = [prob_L_links(N, p, L) for L in Ls]

# Expected number of links
expected_L = p * n_possible

plt.figure(figsize=(10, 5))
plt.bar(Ls, pmf, color=NS_PURPLE, edgecolor='white')
plt.axvline(expected_L, color=NS_ORANGE, linestyle='--', linewidth=2, 
            label=f'⟨L⟩ = {expected_L:.1f}')
plt.xlabel('L (number of links)')
plt.ylabel('P(L)')
plt.title(f'Link Distribution for G(N={N}, p={p})')
plt.legend()
plt.show()

### Expected Number of Links

$$\langle L \rangle = p \cdot \frac{N(N-1)}{2}$$

In [ ]:
N, p = 10, 0.1

n_possible = math.comb(N, 2)
expected_links = p * n_possible

print(f"With N={N} nodes, there are {n_possible} possible edges")
print(f"Expected number of links: ⟨L⟩ = {expected_links}")

### Average Degree

$$\langle k \rangle = \frac{2 \langle L \rangle}{N} = p(N - 1)$$

In [ ]:
N, p = 10, 0.25

expected_links = p * math.comb(N, 2)
avg_degree_v1 = 2 * expected_links / N
avg_degree_v2 = p * (N - 1)

print(f"Average degree (from ⟨L⟩): {avg_degree_v1}")
print(f"Average degree (formula):  {avg_degree_v2}")
assert avg_degree_v1 == avg_degree_v2

---
## 3.4 Degree Distribution

The degree distribution $p_k$ is the probability that a randomly chosen node has degree $k$.

### 3.4.1 Binomial Distribution

The exact degree distribution of a random network follows the binomial distribution:

$$p_k = \binom{N-1}{k} p^k (1 - p)^{N-1-k}$$

Each node can connect to $N-1$ other nodes, and each connection occurs with probability $p$.

In [ ]:
set_seed(42)

N, p = 20, 0.25
n = N - 1  # Each node can connect to N-1 others

# Simulate: sample degrees from binomial distribution
samples = np.random.binomial(n, p, 10000)
values, counts = np.unique(samples, return_counts=True)
empirical_pk = counts / counts.sum()

# Theoretical binomial PMF
ks = np.arange(0, N)
theoretical_pk = binom.pmf(ks, n=n, p=p)

plt.figure(figsize=(10, 5))
plt.bar(values, empirical_pk, color=NS_PURPLE, edgecolor='white', alpha=0.7,
        label='Empirical (10,000 samples)')
plt.plot(ks, theoretical_pk, 'o-', color=NS_GREEN, markersize=8,
         label='Theoretical Binomial')
plt.xlabel('Degree k')
plt.ylabel('$p_k$')
plt.title(f'Degree Distribution: Binomial(n={n}, p={p})')
plt.legend()
plt.show()

### 3.4.2 Poisson Approximation

For **sparse networks** where $\langle k \rangle \ll N$, the degree distribution is well approximated by the Poisson distribution:

$$p_k = e^{-\langle k \rangle} \frac{\langle k \rangle^k}{k!}$$

In [ ]:
set_seed(11)

avg_degree = 5  # ⟨k⟩

# Sample from Poisson
samples = np.random.poisson(avg_degree, 10000)
values, counts = np.unique(samples, return_counts=True)
empirical_pk = counts / counts.sum()

# Theoretical Poisson PMF
ks = np.arange(0, 15)
theoretical_pk = poisson.pmf(ks, mu=avg_degree)

plt.figure(figsize=(10, 5))
plt.bar(values, empirical_pk, color=NS_PURPLE, edgecolor='white', alpha=0.7,
        label='Empirical (10,000 samples)')
plt.plot(ks, theoretical_pk, 'o', color=NS_GREEN, markersize=10,
         label='Theoretical Poisson')
plt.xlabel('Degree k')
plt.ylabel('$p_k$')
plt.title(f'Poisson Degree Distribution: ⟨k⟩ = {avg_degree}')
plt.legend()
plt.show()

### Binomial vs Poisson: When Does Poisson Work?

The Poisson approximation works well when $\langle k \rangle \ll N$. As $N$ increases with fixed $\langle k \rangle$, the binomial converges to Poisson.

In [ ]:
avg_degree = 5
Ns = [10, 100, 1000, 10000]
ks = np.arange(0, 16)

# Poisson (the limit)
pk_poisson = poisson.pmf(ks, mu=avg_degree)

plt.figure(figsize=(10, 6))
plt.bar(ks, pk_poisson, color=NS_PURPLE, edgecolor='white', alpha=0.6,
        label='Poisson')

markers = ['x', '^', 'o', 's']
offsets = np.linspace(-0.2, 0.2, len(Ns))

for N, marker, dx in zip(Ns, markers, offsets):
    n = N - 1
    p = avg_degree / n  # Recover p from ⟨k⟩ = p(N-1)
    pk_binom = binom.pmf(ks, n=n, p=p)
    plt.plot(ks + dx, pk_binom, linestyle='none', marker=marker, markersize=7,
             label=f'Binomial N={N}')

plt.xlabel('Degree k')
plt.ylabel('$p_k$')
plt.xlim(-0.5, 15.5)
plt.ylim(0, 0.2)
plt.title(f'Binomial → Poisson as N increases (⟨k⟩ = {avg_degree})')
plt.legend(frameon=False)
plt.show()

### When Poisson Fails: Dense Networks

When $\langle k \rangle$ is not much smaller than $N$, the Poisson approximation breaks down.

In [ ]:
avg_degree = 80  # High average degree
Ns = [100, 1000, 10000]
ks = np.arange(50, 110)

pk_poisson = poisson.pmf(ks, mu=avg_degree)

plt.figure(figsize=(10, 6))
plt.bar(ks, pk_poisson, color=NS_PURPLE, edgecolor='white', alpha=0.6,
        label='Poisson')

markers = ['^', 'o', 's']
offsets = np.linspace(-0.3, 0.3, len(Ns))

for N, marker, dx in zip(Ns, markers, offsets):
    n = N - 1
    p = avg_degree / n
    pk_binom = binom.pmf(ks, n=n, p=p)
    plt.plot(ks + dx, pk_binom, linestyle='none', marker=marker, markersize=6,
             label=f'Binomial N={N}')

plt.xlabel('Degree k')
plt.ylabel('$p_k$')
plt.xlim(55, 105)
plt.ylim(0, 0.06)
plt.title(f'Poisson fails when ⟨k⟩ = {avg_degree} is NOT ≪ N')
plt.legend(frameon=False)
plt.show()

---
## 3.5 The Evolution of a Random Network

One would expect that the largest component grows gradually from $N_G = 1$ to $N_G = N$ as $\langle k \rangle$ increases. Yet, $N_G/N$ remains zero for small $\langle k \rangle$. Once $\langle k \rangle$ exceeds a critical value, $N_G/N$ increases rapidly—the **giant component** emerges.

### ER Graph Regimes

| Regime | Condition | Description |
|--------|-----------|-------------|
| **Subcritical** | $\langle k \rangle < 1$ | No giant component |
| **Critical** | $\langle k \rangle = 1$ | Critical point |
| **Supercritical** | $\langle k \rangle > 1$ | Giant component emerges |
| **Connected** | $\langle k \rangle > \ln N$ | Graph becomes connected w.h.p. |

In [ ]:
N = 100

regimes = [
    {"label": "subcritical",   "avg_degree": 0.5,  "threshold": "p < 1/N"},
    {"label": "critical",      "avg_degree": 1.0,  "threshold": "p = 1/N"},
    {"label": "supercritical", "avg_degree": 3.75, "threshold": "p > 1/N"},
    {"label": "connected",     "avg_degree": 5.25, "threshold": "p > ln(N)/N"},
]

# Convert ⟨k⟩ to p using ⟨k⟩ = p(N-1)
for r in regimes:
    r["p"] = r["avg_degree"] / (N - 1)

def largest_component_fraction(g):
    """Return fraction of nodes in the largest connected component."""
    largest = max(rx.connected_components(g), key=len)
    return len(largest) / g.num_nodes()

for r in regimes:
    g = rx.undirected_gnp_random_graph(N, r["p"], seed=42)
    gcc = largest_component_fraction(g)
    is_conn = rx.is_connected(g)

    print(
        f"{r['label']:>13} | ⟨k⟩={r['avg_degree']:<4} | p={r['p']:.4f} | "
        f"{r['threshold']:<12} | GCC={gcc:.2f} | connected={is_conn}"
    )

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(14, 4))

for ax, r in zip(axes, regimes):
    g = rx.undirected_gnp_random_graph(N, r["p"], seed=42)
    pos = {i: (float(np.random.rand()), float(np.random.rand())) for i in range(N)}
    gcc_nodes = max(rx.connected_components(g), key=len)
    node_colors = [NS_PURPLE if i in gcc_nodes else 'gray' for i in range(N)]
    
    mpl_draw(g, pos=pos, ax=ax, with_labels=False, node_size=40, 
             node_color=node_colors, edge_color='lightgray', width=0.5)
    ax.set_title(f"{r['label'].title()}\n⟨k⟩ = {r['avg_degree']}")
    ax.axis('off')

plt.suptitle("Random Network Evolution Through Four Regimes", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---

## Box 3.5: Network Evolution in Graph Theory

**Threshold Probabilities for Network Motifs**

As we increase $p$ from 0 to 1, networks develop structure in predictable stages. Each structure (motif) has a **threshold probability** $p_c$:

- Below $p_c$: The motif is **almost surely absent**
- Above $p_c$: The motif is **almost surely present**

| Motif | Threshold $p_c$ | For $N=100$ |
|-------|-----------------|-------------|
| **Edge** (2 nodes connected) | $N^{-2}$ | 0.0001 |
| **Path of length 2** (wedge) | $N^{-3/2}$ | 0.001 |
| **3-star** | $N^{-4/3}$ | 0.0046 |
| **4-star** | $N^{-5/4}$ | 0.0056 |
| **Triangle** | $N^{-1}$ | 0.01 |
| **Complete graph $K_4$** | $N^{-2/3}$ | 0.0215 |
| **Giant component** | $N^{-1}$ | 0.01 |
| **Fully connected** | $\ln(N)/N$ | 0.046 |

**Key Insight**: Larger or denser motifs require higher $p$ to appear. The mathematical theory predicts sharp phase transitions at these thresholds.

In [ ]:
# Motif detection functions for testing thresholds

def exists_edge(g) -> bool:
    """Check if at least one edge exists in the graph."""
    return g.num_edges() > 0

def exists_wedge_len2(g) -> bool:
    """Check if path of length 2 (wedge) exists: node with degree >= 2."""
    return any(g.degree(n) >= 2 for n in g.node_indices())

def exists_3star(g) -> bool:
    """Check if 3-star exists: node with degree >= 3."""
    return any(g.degree(n) >= 3 for n in g.node_indices())

def exists_4star(g) -> bool:
    """Check if 4-star exists: node with degree >= 4."""
    return any(g.degree(n) >= 4 for n in g.node_indices())

def exists_triangle(g) -> bool:
    """Check if any triangle exists in the graph."""
    for n in g.node_indices():
        neighbors = list(g.neighbors(n))
        for i, n1 in enumerate(neighbors):
            for n2 in neighbors[i+1:]:
                if g.has_edge(n1, n2):
                    return True
    return False

def exists_K4(g) -> bool:
    """Check if complete graph K4 exists (4 fully connected nodes)."""
    from itertools import combinations
    for nodes in combinations(g.node_indices(), 4):
        edges = [(nodes[i], nodes[j]) for i in range(4) for j in range(i+1, 4)]
        if all(g.has_edge(u, v) for u, v in edges):
            return True
    return False

In [ ]:
# Test motifs across different average degrees
N = 100
z_values = [0.5, 1.0, 2.0, 3.0, 5.0, 10.0]  # average degrees to test
n_trials = 100

motifs = [
    ("Edge", exists_edge),
    ("Wedge (len 2)", exists_wedge_len2),
    ("3-star", exists_3star),
    ("4-star", exists_4star),
    ("Triangle", exists_triangle),
    ("K4", exists_K4),
]

print(f"{'Motif':<15} | " + " | ".join(f"z={z:<4}" for z in z_values))
print("-" * 70)

for name, test_fn in motifs:
    probs = []
    for z in z_values:
        p = z / (N - 1)
        count = sum(test_fn(rx.undirected_gnp_random_graph(N, p)) for _ in range(n_trials))
        probs.append(count / n_trials)
    print(f"{name:<15} | " + " | ".join(f"{prob:>5.0%}" for prob in probs))

---

## 3.7 Real Networks are Supercritical

Two predictions of random network theory are of direct importance for real networks:

1. Once the average degree exceeds $\langle k \rangle = 1$, a **giant component** should emerge that contains a finite fraction of all nodes. Hence only for $\langle k \rangle > 1$ do nodes organize themselves into a recognizable network.

2. For $\langle k \rangle > \ln N$, all components are absorbed by the giant component, resulting in a **single connected network**.

In [ ]:
# Load Internet topology data
file = 'internet.edgelist.txt'
dataset = DATADIR / file

with dataset.open() as f:
    data = [tuple(l.strip().split('\t')) for l in f.readlines()]

edges = [tuple(map(int, x)) for x in data]
nodes = list(set(itertools.chain.from_iterable(data)))

G_internet = rx.PyGraph()
G_internet.add_nodes_from(nodes)
G_internet.add_edges_from_no_data(edges)

N = len(nodes)
L = len(edges)
avg_degree = 2 * L / N
lnN = np.log(N)

print('Internet Data')
print(f"Nodes (N):     {N}")
print(f"Edges (L):     {L}")
print(f"⟨k⟩:           {avg_degree:.2f}")
print(f"ln(N):         {lnN:.2f}")
print(f"⟨k⟩ > 1?       {avg_degree > 1}")
print(f"⟨k⟩ > ln(N)?   {avg_degree > lnN}")

---

## 3.8 Small Worlds

The "small world" phenomenon means that the average path length depends **logarithmically** on the system size. That is, the average distance $\langle d \rangle$ between two nodes scales as $\ln N$ rather than $N$.

### Counting Nodes at Distance d

The expected number of nodes within distance $d$ from a starting node (in a tree-like approximation):

$$N(d) \approx 1 + \langle k \rangle + \langle k \rangle^2 + \cdots + \langle k \rangle^d = \frac{\langle k \rangle^{d+1} - 1}{\langle k \rangle - 1}$$

In [ ]:
def nodes_within_distance(k: float, d: int) -> float:
    """
    Expected number of nodes within distance d in the branching approximation:
        1 + k + k^2 + ... + k^d
    """
    if np.isclose(k, 1.0):
        return d + 1
    return (k**(d + 1) - 1) / (k - 1)

# Example: estimate N given avg_degree and diameter
avg_degree = 10
diameter = 5

estimated_N = nodes_within_distance(avg_degree, diameter)
print(f"With ⟨k⟩={avg_degree} and d_max={diameter}:")
print(f"Estimated N = {estimated_N:.0f}")

### Deriving Average Distance

Since $N(d)$ cannot exceed $N$, we set $N(d_{max}) \approx N$. For $\langle k \rangle \gg 1$:

$$\langle k \rangle^{d_{max}} \approx N$$

Taking logarithms:

$$d_{max} \approx \frac{\ln N}{\ln \langle k \rangle}$$

This better approximates the **average distance** $\langle d \rangle$ than the diameter, because the diameter is dominated by extreme paths while $\langle d \rangle$ averages over all node pairs:

$$\langle d \rangle \approx \frac{\ln N}{\ln \langle k \rangle}$$

In [ ]:
def avg_degree_fn(G):
    """For an undirected graph, ⟨k⟩ = 2L/N"""
    return 2 * G.num_edges() / G.num_nodes()

def graph_diameter(G):
    """Compute diameter using all-pairs shortest paths."""
    apsp = rx.all_pairs_dijkstra_path_lengths(G, edge_cost_fn=lambda _: 1)
    return max(dist for mapping in apsp.values() for dist in mapping.values())

# Create a random graph to test
N, p = 1000, 0.01
G = rx.undirected_gnp_random_graph(N, p, seed=67)

avg_degree = avg_degree_fn(G)
diameter = graph_diameter(G)
empirical_avg_d = rx.unweighted_average_shortest_path_length(G)
predicted_avg_d = np.log(G.num_nodes()) / np.log(avg_degree)

print(f"Number of nodes: {G.num_nodes():,}")
print(f"Average degree: {avg_degree:.3f}")
print(f"Diameter (max shortest path): {diameter}")
print(f"Measured ⟨d⟩: {empirical_avg_d:.3f}")
print(f"Predicted ⟨d⟩ ≈ ln(N)/ln⟨k⟩: {predicted_avg_d:.3f}")

The small world formula allows us to estimate path lengths in large networks without computing all pairwise distances. This is the mathematical basis for phenomena like "six degrees of separation" — using knowledge of the average degree and network size, we can predict that social networks have short paths connecting any two people.

---

## 3.9 Clustering Coefficient

The degree of a node contains no information about the relationship between a node's neighbors. The **local clustering coefficient** $C_i$ measures the density of links in node $i$'s immediate neighborhood:

- $C_i = 0$ means there are no links between node $i$'s neighbors
- $C_i = 1$ means each of node $i$'s neighbors link to each other

### Expected Clustering in Random Networks

In a random network, the probability that two of node $i$'s neighbors link to each other is $p$. The expected number of edges among neighbors:

$$\langle L_i \rangle = p \frac{k_i(k_i - 1)}{2}$$

Thus the expected local clustering coefficient is:

$$\mathbb{E}[C_i] = p \approx \frac{\langle k \rangle}{N}$$

In [ ]:
N, p = 1000, 0.001

# Expected values for random networks
expected_links = p * (N * (N - 1)) / 2
expected_degree = p * (N - 1)
expected_C = expected_degree / N

print(f"Expected links: {expected_links:.1f}")
print(f"Expected degree ⟨k⟩: {expected_degree:.3f}")
print(f"Expected clustering coefficient E[C]: {expected_C:.6f}")
print(f"Note: E[C] ≈ p = {p}")

# Verify E[C] ≈ p
assert np.isclose(expected_C, p, rtol=0.01)

### Key Predictions

1. For fixed $\langle k \rangle$, the larger the network, the smaller the clustering coefficient ($C_i \sim 1/N$)
2. **In a random network, the local clustering coefficient is independent of the node's degree**

In [ ]:
def local_clustering_coef(G, i: int) -> float:
    """C_i = (# edges among neighbors of i) / (k_i choose 2) for an undirected graph."""
    nbrs = list(G.neighbors(i))
    k = len(nbrs)
    if k < 2:
        return 0.0

    edges_between = sum(G.has_edge(u, v) for u, v in combinations(nbrs, 2))
    possible = k * (k - 1) / 2
    return edges_between / possible

### Comparing C(k) in Real Networks

Let's load a protein interaction network and compute how clustering varies with degree:

In [ ]:
# Load protein interaction network
file = 'protein.edgelist.txt'
dataset = DATADIR / file

with dataset.open() as f:
    data = [tuple(l.strip().split('\t')) for l in f.readlines()]

edges = [tuple(map(int, x)) for x in data]
nodes = list(set(itertools.chain.from_iterable(data)))

G_protein = rx.PyGraph()
G_protein.add_nodes_from(nodes)
G_protein.add_edges_from_no_data(edges)

print(f"Protein interaction network:")
print(f"  Nodes: {G_protein.num_nodes()}")
print(f"  Edges: {G_protein.num_edges()}")

In [ ]:
from collections import defaultdict

# Compute per-node degree and C_i
nodes_list = list(G_protein.node_indices())
deg = np.array([G_protein.degree(i) for i in nodes_list], dtype=int)
Ci = np.array([local_clustering_coef(G_protein, i) for i in nodes_list], dtype=float)

# Network average clustering <C> vs expected <C> for random network
N_prot = G_protein.num_nodes()
L_prot = G_protein.num_edges()
p_equiv = 2 * L_prot / (N_prot * (N_prot - 1))  # equivalent p for same density
C_avg = Ci.mean()
C_exp = p_equiv

print(f"Average clustering ⟨C⟩: {C_avg:.4f}")
print(f"Expected for random network E[C] = p: {C_exp:.6f}")
print(f"Real network has {C_avg/C_exp:.0f}x higher clustering!")

# Compute C(k): average C_i for nodes with the same degree k
by_k = defaultdict(list)
for k, c in zip(deg, Ci):
    by_k[k].append(c)

ks = np.array(sorted(by_k.keys()), dtype=int)
Ck = np.array([np.mean(by_k[k]) for k in ks], dtype=float)

# Plot: C(k) vs k on log-log axes
mask = (ks > 0) & (Ck > 0)

plt.figure(figsize=(6, 4))
plt.scatter(ks[mask], Ck[mask], s=18, alpha=0.9, color=NS_PURPLE, label='C(k) measured')
plt.axhline(C_avg, linewidth=2, color=NS_GREEN, label=f'⟨C⟩ = {C_avg:.3f}')
plt.axhline(C_exp, linewidth=2, color=NS_ORANGE, label=f'E[C] random = {C_exp:.4f}')

plt.xscale("log")
plt.yscale("log")
plt.xlabel("Degree k")
plt.ylabel("C(k)")
plt.title("Clustering vs Degree in Protein Network")
plt.legend()
plt.tight_layout()
plt.show()

---

## Summary

In this chapter, we explored the **Erdős-Rényi random network model** and its key properties:

**3.2 The Random Network Model**
- Two equivalent definitions: $G(N, L)$ fixes edge count, $G(N, p)$ fixes connection probability
- The $G(N, p)$ model is more analytically tractable

**3.3 Number of Links**
- Expected links: $⟨L⟩ = p \cdot \binom{N}{2}$
- Links follow a binomial distribution with very small variance for large $N$

**3.4 Degree Distribution**
- Follows a binomial distribution: $p_k = \binom{N-1}{k} p^k (1-p)^{N-1-k}$
- Approximated by Poisson distribution for large $N$: $p_k \approx e^{-⟨k⟩} \frac{⟨k⟩^k}{k!}$
- Average degree: $⟨k⟩ = p(N-1)$

**3.5 Evolution of a Random Network**
- As $⟨k⟩$ increases, the network passes through four distinct regimes:
  - **Subcritical** ($⟨k⟩ < 1$): Small, isolated clusters
  - **Critical** ($⟨k⟩ = 1$): Giant component emerges
  - **Supercritical** ($⟨k⟩ > 1$): Giant component dominates
  - **Connected** ($⟨k⟩ > \ln N$): Network becomes fully connected

**Box 3.5: Network Evolution in Graph Theory**
- Sharp phase transitions occur at specific threshold probabilities
- Each network motif (edge, triangle, K4, etc.) has a characteristic threshold

**3.7 Real Networks are Supercritical**
- Real networks consistently have $⟨k⟩ > 1$ and often $⟨k⟩ > \ln N$
- This explains why real networks form connected structures

**3.8 Small Worlds**
- Average distance scales logarithmically: $⟨d⟩ \approx \frac{\ln N}{\ln ⟨k⟩}$
- Enables "six degrees of separation" in large social networks

**3.9 Clustering Coefficient**
- Local clustering $C_i$ measures link density among a node's neighbors
- In random networks: $\mathbb{E}[C_i] = p \approx \langle k \rangle / N$
- Clustering is independent of degree in random networks
- Real networks typically have much higher clustering than random networks